### Random Forest Classifier WandB Sweep

In [1]:
import pandas as pd

In [2]:
idle_time_data = pd.read_csv('../data/df_points/df_points_18_21_class.csv')

In [3]:
from sklearn.preprocessing import StandardScaler

TargetVariable = ['idle_time_class']
Predictors = ['bike_id', 'lat', 'lng', 'temp', 'rain', 'snow', 'dt_start', 'hex_enc', 'start_min', 'month', 'day']

X = idle_time_data[Predictors]
y = idle_time_data[TargetVariable]

PredictorScaler=StandardScaler()
PredictorScalerFit=PredictorScaler.fit(X)
X=PredictorScalerFit.transform(X)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, shuffle=False)
### shuffle/inorder train and test sets:
# , shuffle=False
# , random_state=42

#### ggf irgendwann ts_cv zur cross validation

from sklearn.model_selection import TimeSeriesSplit

ts_cv = TimeSeriesSplit(n_splits=4,max_train_size=10000)

all_splits = list(ts_cv.split(X_train, y_train))
train_0, test_0 = all_splits[0]

In [4]:
from sklearn.model_selection import TimeSeriesSplit

ts_cv = TimeSeriesSplit(n_splits=8)
ts_cv

TimeSeriesSplit(gap=0, max_train_size=None, n_splits=8, test_size=None)

In [6]:
sweep_configuration_rfc = {
    "project": "RandomForestClass",
    "name": "RFC-sweep-split-inorder",
    "metric": {"name": "accuracy", "goal": "maximize"},
    "method": "random",
    "parameters": {
        "n_estimators": {
            "values": [8, 16, 32, 64, 128, 256]
        },
        "criterion": {
            "values": ['entropy','gini']
        },
        "max_depth": {
            "values": [2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 120, 140, 160, 180, 200, None]
        },
        "bootstrap": {
            "values": [True, False]
        },
        "max_features": {
            "values": ['auto', 'sqrt', 'log2']
        },
        "min_samples_leaf": {
            "values": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 16, 18, 20, 25, 30]
        },
        "min_samples_split": {
            "values": [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 15, 18, 20, 25, 30]
        }
    }
}

In [7]:
from sklearn.metrics import zero_one_loss, accuracy_score
from sklearn.model_selection import cross_validate, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import wandb
import wandb

def my_train_func():
    wandb.init()

    _n_estimators = wandb.config.n_estimators
    _criterion = wandb.config.criterion
    _max_depth = wandb.config.max_depth
    _bootstrap = wandb.config.bootstrap
    _max_features = wandb.config.max_features
    _min_samples_leaf = wandb.config.min_samples_leaf
    _min_samples_split = wandb.config.min_samples_split

    model = RandomForestClassifier(n_estimators=_n_estimators,
                                   criterion=_criterion,
                                   max_depth=_max_depth,
                                   bootstrap=_bootstrap,
                                   max_features=_max_features,
                                   min_samples_leaf=_min_samples_leaf,
                                   min_samples_split=_min_samples_split)

    model.fit(X_train,y_train.ravel())
    y_pred = model.predict(X_test)

    score_training = model.score(X_train, y_train.ravel())
    score_validation = model.score(X_test, y_test.ravel())

    acc = zero_one_loss(y_test.ravel(), y_pred.ravel())
    loss = accuracy_score(y_test.ravel(), y_pred.ravel())

    wandb.sklearn.plot_feature_importances(model, Predictors)

    wandb.log({"accuracy": acc})
    wandb.log({"conf_matrix": wandb.plot.confusion_matrix(y_true=y_test.ravel(), preds=y_pred.ravel())})
    wandb.log({"feature_imp": wandb.sklearn.plot_feature_importances(model, Predictors)})
    wandb.log({"loss": loss})
    wandb.log({"score_training":score_training,"score_validation":score_validation})


In [12]:
def my_train_func_cv():
    wandb.init()

    _n_estimators = wandb.config.n_estimators
    _criterion = wandb.config.criterion
    _max_depth = wandb.config.max_depth
    _bootstrap = wandb.config.bootstrap
    _max_features = wandb.config.max_features
    _min_samples_leaf = wandb.config.min_samples_leaf
    _min_samples_split = wandb.config.min_samples_split

    model = RandomForestClassifier(n_estimators=_n_estimators,
                                   criterion=_criterion,
                                   max_depth=_max_depth,
                                   bootstrap=_bootstrap,
                                   max_features=_max_features,
                                   min_samples_leaf=_min_samples_leaf,
                                   min_samples_split=_min_samples_split)

    cv_res = cross_val_score(model,X_train,y_train,cv = ts_cv, scoring='accuracy')

    #score_training = model.score(X_train, y_train)
    #score_validation = model.score(X_test, y_test)

    dict = {"acc1": cv_res[0],
            "acc2": cv_res[1],
            "acc3": cv_res[2],
            "acc4": cv_res[3],
            "acc5": cv_res[4],
            "acc6": cv_res[5],
            "acc7": cv_res[6],
            "acc8": cv_res[7]
            }

    wandb.log({"cv_accuracy": dict})
    #wandb.log({"score_training":score_training,"score_validation":score_validation})


In [13]:
# INIT SWEEP
sweep_id_rfc = wandb.sweep(sweep_configuration_rfc, project="test")
# RUN SWEEP
wandb.agent(sweep_id_rfc, function=my_train_func_cv)


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: xinzo5rj
Sweep URL: https://wandb.ai/jonathanweske/test/sweeps/xinzo5rj


wandb: Agent Starting Run: 9bsfu89y with config:
wandb: 	bootstrap: False
wandb: 	criterion: gini
wandb: 	max_depth: 2
wandb: 	max_features: sqrt
wandb: 	min_samples_leaf: 3
wandb: 	min_samples_split: 6
wandb: 	n_estimators: 8


/usr/local/Caskroom/miniconda/base/envs/geobike/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:681: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/Caskroom/miniconda/base/envs/geobike/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:681: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/Caskroom/miniconda/base/envs/geobike/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:681: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/Caskroom/miniconda/b

wandb: Agent Starting Run: 1y0o2isn with config:
wandb: 	bootstrap: True
wandb: 	criterion: gini
wandb: 	max_depth: 9
wandb: 	max_features: log2
wandb: 	min_samples_leaf: 2
wandb: 	min_samples_split: 13
wandb: 	n_estimators: 32


/usr/local/Caskroom/miniconda/base/envs/geobike/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:681: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/Caskroom/miniconda/base/envs/geobike/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:681: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/Caskroom/miniconda/base/envs/geobike/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:681: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/Caskroom/miniconda/b

#### CV cross_validate()
Mean Absolute Error:     0.672 +/- 0.019
Root Mean Squared Error: 1.057 +/- 0.016

#### cross_val_score
[0.49957698 0.53019349 0.54551058 0.49902538]
Accuracy of Model with Cross Validation is: 51.85766080195356